In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
#PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/mythr/Documents/Github/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
PATH = "C:/Users/n9455647/Documents/Github/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
#!pip install lime
#!pip install shap
#!pip install pandas==0.19.2
!pip install xgboost==1.0.0

  Using cached xgboost-1.0.0-py3-none-win_amd64.whl (24.6 MB)
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.1.1
    Uninstalling xgboost-1.1.1:
      Successfully uninstalled xgboost-1.1.1


In [3]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory
import stability as st #Nogueira, Sechidis, Brown.

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import lime
import lime.lime_tabular
from lime import submodular_pick;

import shap

import warnings
warnings.filterwarnings('ignore')

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def imp_df(column_names, importances):
        df = pd.DataFrame({'feature': column_names,
                       'feature_importance': importances}) \
           .sort_values('feature_importance', ascending = False) \
           .reset_index(drop = True)
        return df

# plotting a feature importance dataframe (horizontal barchart)
def var_imp_plot(imp_df, title, num_feat):
        imp_df.columns = ['feature', 'feature_importance']
        b= sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df.head(num_feat), orient = 'h', palette="Blues_r")

In [5]:
def generate_global_explanations(train_X,train_Y, cls, feature_combiner):
    
    print("The number of testing instances is ",len(train_Y))
    print("The total number of columns is",train_X.shape[1]);
    print("The total accuracy is ",cls.score(train_X,train_Y));
       
    sns.set(rc={'figure.figsize':(10,10), "font.size":18,"axes.titlesize":18,"axes.labelsize":18})
    sns.set
    feat_names = feature_combiner.get_feature_names()
    base_imp = imp_df(feat_names, cls.feature_importances_)
    base_imp.head(15)
    var_imp_plot(base_imp, 'Feature importance using XGBoost', 15)
    return base_imp

In [6]:
from lime import submodular_pick
def generate_lime_explanations(explainer,test_xi, cls,test_y, submod=False, test_all_data=None, max_feat = 10):
    
    #print("Actual value ", test_y)
    exp = explainer.explain_instance(test_xi, 
                                 cls.predict_proba, num_features=max_feat, labels=[0,1])
    
    return exp
        
    if submod==True:
        sp_obj=submodular_pick.SubmodularPick(explainer, test_all_data, cls.predict_proba, 
                                      sample_size=20, num_features=num_features,num_exps_desired=4)
        [exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];

In [7]:
def create_samples(shap_explainer, iterations, row, features, top = 10):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    for j in range(iterations):
        
        #if shap_type == "kernel":
        #    shap_explainer = shap.KernelExplainer(cls.predict, trainingsample)
        #elif shap_type == "tree":
        #    shap_explainer = shap.TreeExplainer(cls)
        #elif shap_type == "deep":
        #    shap_explainer = shap.DeepExplainer(cls, background)
        
        #print(X_test_frame.loc[row])
        shap_values = shap_explainer.shap_values(row)
        #print(shap_values)

        importances = []
        
        if type(shap_explainer) == shap.explainers.kernel.KernelExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
                
        elif type(shap_explainer) == shap.explainers.tree.TreeExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][i]
                abs_val = abs(shap_values[0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        elif type(shap_explainer) == shap.explainers.deep.DeepExplainer:
            for i in range(length):
                feat = features[i]
                shap_val = shap_values[0][0][i]
                abs_val = abs(shap_values[0][0][i])
                entry = (feat, shap_val, abs_val)
                importances.append(entry)
        
        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        exp.append(importances)

        rel_feat = []

        for i in range(top):
            feat = importances[i]
            if feat[2] > 0:
                rel_feat.append(feat)
                
        rel_exp.append(rel_feat)
        
    return exp, rel_exp

In [8]:
def dispersal(weights, features):
    feat_len = len(features)
    #print(feat_len)
    weights_by_feat = []
       
    for i in list(range(feat_len)):
        feat_weight = []
        for iteration in weights:
            feat_weight.append(iteration[i])
        weights_by_feat.append(feat_weight)
        
    #for iteration in weights:
     #   for val in iteration:
      #      idx = iteration.index(val)
       #     print(idx)
        #    weights_by_feat[idx].append(val)
    
    dispersal = []
    
    for each in weights_by_feat:
        if np.mean(each) == 0:
            dispersal.append(0)
        #print(each)
        else:
            var = (np.std(each))**2/abs(np.mean(each))
            dispersal.append(var)
    
    return dispersal

In [9]:
dataset_ref = "bpic2012"
params_dir = PATH + "params"
results_dir = "results"
bucket_method = "prefix"
cls_encoding = "agg"
cls_method = "xgboost"

gap = 1
n_iter = 1

method_name = "%s_%s"%(bucket_method, cls_encoding)

generate_samples = True
generate_lime = True
generate_kernel_shap = False
generate_model_shap = True

sample_size = 2
exp_iter = 10
max_feat = 10

dataset_ref_to_datasets = {
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"]
    #"insurance": ["insurance_activity", "insurance_followup"],
    #"sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

datasets

['bpic2012_accepted']

In [10]:
if generate_samples:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                pipeline_path = os.path.join(PATH, "%s/%s_%s/pipelines/pipeline_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                bucketer_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/bucketer_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))

                predictor = joblib.load(pipeline_path)
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                bucketer = joblib.load(bucketer_path)

                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                X_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_test_path = os.path.join(PATH, "%s/%s_%s/test_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                with open (X_test_path, 'rb') as f:
                    dt_test_bucket = pickle.load(f)
                with open (Y_test_path, 'rb') as f:
                    test_y = pickle.load(f)

                #import previous results from predictions
                tn_path = os.path.join(PATH, "%s/%s_%s/instances/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/instances/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/instances/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/instances/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                instances.append(tn_list)
                instances.append(tp_list)
                instances.append(fn_list)
                instances.append(fp_list)

                #choose instances from the event log to explain, based on different prefix lengths
                sample_instances = []
                
                for each in instances:
                    sample = []
                    #Find instances of different prediction probabilities
                    prs = [0.5, 0.6, 0.7, 0.8, 0.9, 1.1]
                    for i in list(range(len(prs)-1)):
                        low = prs[i]
                        high = prs[i+1]
                        ins = [d for d in each if (d['proba'] >= low) & (d['proba'] < high)]
                        if len(ins) >= sample_size:
                            sample.extend(random.sample(ins, k=sample_size))
                        else:
                            sample.extend(ins)
                    sample = sorted(sample, key = lambda i: (i['proba'], i['nr_events']))
                    sample_instances.append(sample)

                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1
Bucket 2
Bucket 3
Bucket 4
Bucket 5
Bucket 6
Bucket 7
Bucket 8
Bucket 9
Bucket 10
Bucket 11
Bucket 12
Bucket 13
Bucket 14
Bucket 15
Bucket 16
Bucket 17
Bucket 18
Bucket 19
Bucket 20
Bucket 21
Bucket 22
Bucket 23
Bucket 24
Bucket 25
Bucket 26
Bucket 27
Bucket 28
Bucket 29
Bucket 30
Bucket 31
Bucket 32
Bucket 33
Bucket 34
Bucket 35
Bucket 36
Bucket 37
Bucket 38
Bucket 39
Bucket 40


In [11]:
if generate_model_shap:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                #X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                #Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                #with open (X_train_path, 'rb') as f:
                #    dt_train_bucket = pickle.load(f)
                #with open (Y_train_path, 'rb') as f:
                #    train_y = pickle.load(f)
                #with open (X_test_path, 'rb') as f:
                #    dt_test_bucket = pickle.load(f)
                #with open (Y_test_path, 'rb') as f:
                #    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

                #create explainers now that can be passed later
                tree_explainer = shap.TreeExplainer(cls)

                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    
                    for instance in category:
                        instance_no += 1    
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                
                        group = instance['input']

                        print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
                        test_x=np.transpose(test_x_group[0])

                        print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get Tree SHAP explanations for instance
                        exp, rel_exp = create_samples(tree_explainer, exp_iter, test_x.reshape(-1, len(test_x)), feat_list, top = max_feat)

                        feat_pres = []
                        feat_weights = []

                        for iteration in rel_exp:
                            print("Tree SHAP Iteration", rel_exp.index(iteration))
                            #print(iteration)

                            #Stability by index
                            presence_list = [0]*len(feat_list)
                            #stability by weight
                            weights = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in iteration:
                                    if each in explanation[0]:
                                        presence_list[list_idx] = 1
                                        weights[list_idx] = explanation[1]

                            feat_pres.append(presence_list)
                            feat_weights.append(weights)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['tree_shap_stability'] = stability
                        
                        norm_var = dispersal(feat_weights, feat_list)
                        avg_dispersal = np.mean(norm_var)
                        print ("Dispersal of feature importance:", round(avg_dispersal, 2))
                        instance['shap_weights_dispersal'] = norm_var
                        instance['shap_importance_stability'] = avg_dispersal

                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1
Category 1 of 4 . Testing 1 of 10 .
(1, 18) 0 0
Generating local Explanations for 205921
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 1 of 4 . Testing 2 of 10 .
(1, 18) 0 0
Generating local Explanations for 211158
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 1 of 4 . Testing 3 of 10 .
(1, 18) 0 0
Generating local Explanations for 211883
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree

Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 3 of 4 . Testing 5 of 10 .
(1, 18) 1 0
Generating local Explanations for 206198
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 3 of 4 . Testing 6 of 10 .
(1, 18) 1 0
Generating local Explanations for 211074
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Catego

Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 1 of 4 . Testing 9 of 10 .
(2, 18) 0 0
Generating local Explanations for 205340_2
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 1 of 4 . Testing 10 of 10 .
(2, 18) 0 0
Generating local Explanations for 213085_2
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
C

Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 4 of 4 . Testing 3 of 10 .
(2, 18) 0 1
Generating local Explanations for 211573_2
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 4 of 4 . Testing 4 of 10 .
(2, 18) 0 1
Generating local Explanations for 206706_2
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Ca

Generating local Explanations for 208118_3
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 2 of 4 . Testing 7 of 10 .
(3, 18) 1 1
Generating local Explanations for 208607_3
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 2 of 4 . Testing 8 of 10 .
(3, 18) 1 1
Generating local Explanations for 211606_3
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 

...

Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 1 of 4 . Testing 9 of 10 .
(38, 18) 0 0
Generating local Explanations for 210866_38
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 1 of 4 . Testing 10 of 10 .
(38, 18) 0 0
Generating local Explanations for 206411_38
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0

Generating local Explanations for 206925_38
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 4 of 4 . Testing 4 of 10 .
(38, 18) 0 1
Generating local Explanations for 204793_38
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 4 of 4 . Testing 5 of 10 .
(38, 18) 0 1
Generating local Explanations for 205676_38
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stabil

Generating local Explanations for 211074_39
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 2 of 4 . Testing 7 of 10 .
(39, 18) 1 1
Generating local Explanations for 210272_39
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 2 of 4 . Testing 8 of 10 .
(39, 18) 1 1
Generating local Explanations for 204751_39
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stabil

Category 1 of 4 . Testing 1 of 10 .
(40, 18) 0 0
Generating local Explanations for 205846_40
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 1 of 4 . Testing 2 of 10 .
(40, 18) 0 0
Generating local Explanations for 204993_40
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 1 of 4 . Testing 3 of 10 .
(40, 18) 0 0
Generating local Explanations for 205794_40
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
T

Generating local Explanations for 208619_40
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 3 of 4 . Testing 6 of 8 .
(40, 18) 1 0
Generating local Explanations for 205468_40
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stability: 1.0
Dispersal of feature importance: 0.0
Category 3 of 4 . Testing 7 of 8 .
(40, 18) 1 0
Generating local Explanations for 205535_40
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Tree SHAP Iteration 0
Stabilit

In [12]:
if generate_lime:

    for dataset_name in datasets:
        
        dataset_manager = DatasetManager(dataset_name)
        
        for ii in range(n_iter):
            num_buckets = range(len([name for name in os.listdir(os.path.join(PATH,'%s/%s_%s/models'% (dataset_ref, cls_method, method_name)))]))
            
            for bucket in list(num_buckets):
                bucketID = bucket+1
                print ('Bucket', bucketID)

                #import everything needed to sort and predict
                feat_comb_path = os.path.join(PATH, "%s/%s_%s/bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls_path = os.path.join(PATH, "%s/%s_%s/models/cls_bucket_%s.joblib" % (dataset_ref, cls_method, method_name, bucketID))
                cls = joblib.load(cls_path)
                feature_combiner = joblib.load(feat_comb_path)
                
                #import data for bucket
                X_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_prefixes.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                Y_train_path = os.path.join(PATH, "%s/%s_%s/train_data/bucket_%s_labels.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                
                with open (X_train_path, 'rb') as f:
                    dt_train_bucket = pickle.load(f)
                with open (Y_train_path, 'rb') as f:
                    train_y = pickle.load(f)
                #with open (X_test_path, 'rb') as f:
                #    dt_test_bucket = pickle.load(f)
                #with open (Y_test_path, 'rb') as f:
                #    test_y = pickle.load(f)

                #import previously identified samples
                tn_path = os.path.join(PATH, "%s/%s_%s/samples/true_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                tp_path = os.path.join(PATH, "%s/%s_%s/samples/true_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fn_path = os.path.join(PATH, "%s/%s_%s/samples/false_neg_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))
                fp_path = os.path.join(PATH, "%s/%s_%s/samples/false_pos_bucket_%s_.pickle" % (dataset_ref, cls_method, method_name, bucketID))

                sample_instances = []

                with open (tn_path, 'rb') as f:
                    tn_list = pickle.load(f)
                with open (tp_path, 'rb') as f:
                    tp_list = pickle.load(f)
                with open (fn_path, 'rb') as f:
                    fn_list = pickle.load(f)
                with open (fp_path, 'rb') as f:
                    fp_list = pickle.load(f)

                #save results to a list
                sample_instances.append(tn_list)
                sample_instances.append(tp_list)
                sample_instances.append(fn_list)
                sample_instances.append(fp_list)

                #get the training data as a matrix
                trainingdata = feature_combiner.fit_transform(dt_train_bucket);
                #importance = generate_global_explanations(trainingdata,train_y, cls, feature_combiner)

                #explain the chosen instances and find the stability score
                cat_no = 0
                for category in sample_instances:
                    cat_no += 1
                    instance_no = 0
                    for instance in category:
                        instance_no += 1
                        
                        print("Category", cat_no, "of", len(sample_instances), ". Testing", instance_no, "of", len(category), ".")
                        
                        group = instance['input']
                        
                        #create explainer now that can be passed later
                        class_names=['regular','deviant']# regular is 0, deviant is 1, 0 is left, 1 is right
                        lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata,
                                          feature_names = feature_combiner.get_feature_names(),
                                          class_names=class_names, discretize_continuous=True)

                        #print(group.shape,instance['actual'], instance['predicted'])
                        test_x_group= feature_combiner.fit_transform(group) 
                        test_x=np.transpose(test_x_group[0])

                        print('Generating local Explanations for', instance['caseID'])

                        feat_list = feature_combiner.get_feature_names()

                        #Get lime explanations for instance
                        feat_pres = []
                        feat_weights = []

                        for iteration in list(range(exp_iter)):
                            print("Run", iteration)
                            
                            lime_exp = generate_lime_explanations(lime_explainer, test_x, cls, instance['actual'], max_feat = max_feat)

                            presence_list = [0]*len(feat_list)
                            weights = [0]*len(feat_list)

                            for each in feat_list:
                                list_idx = feat_list.index(each)
                                #print ("Feature", list_idx)
                                for explanation in lime_exp.as_list():
                                    if each in explanation[0]:
                                        #if lime_exp.as_list().index(explanation) < max_feat:
                                            presence_list[list_idx] = 1
                                            weights[list_idx] = explanation[1]

                            feat_pres.append(presence_list)
                            feat_weights.append(weights)

                        stability = st.getStability(feat_pres)
                        print ("Stability:", round(stability,2))
                        instance['lime_stability'] = stability
                        
                        norm_var = dispersal(feat_weights, feat_list)
                        avg_dispersal = np.mean(norm_var)
                        print ("Dispersal of feature importance:", round(avg_dispersal, 2))
                        instance['lime_weights_dispersal'] = norm_var
                        instance['lime_importance_stability'] = avg_dispersal
                        
                #Save dictionaries updated with stability scores
                with open(tn_path, 'wb') as f:
                    pickle.dump(sample_instances[0], f)
                with open(tp_path, 'wb') as f:
                    pickle.dump(sample_instances[1], f)
                with open(fn_path, 'wb') as f:
                    pickle.dump(sample_instances[2], f)
                with open(fp_path, 'wb') as f:
                    pickle.dump(sample_instances[3], f)

Bucket 1
Category 1 of 4 . Testing 1 of 10 .
Generating local Explanations for 205921
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.52
Dispersal of feature importance: 0.01
Category 1 of 4 . Testing 2 of 10 .
Generating local Explanations for 211158
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.42
Dispersal of feature importance: 0.03
Category 1 of 4 . Testing 3 of 10 .
Generating local Explanations for 211883
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.45
Dispersal of feature importance: 0.02
Category 1 of 4 . Testing 4 of 10 .
Generating local Explanations for 205715
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.48
Dispersal of feature importance: 0.03
Category 1 of 4 . Testing 5 of 10 .
Generating local Explanations for 204565
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.5
Dispersal of feature importance: 0.54
Category 1 of 4 . Testing 6 of 10 .
G

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.46
Dispersal of feature importance: 0.07
Category 1 of 4 . Testing 5 of 10 .
Generating local Explanations for 207542_2
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.51
Dispersal of feature importance: 0.04
Category 1 of 4 . Testing 6 of 10 .
Generating local Explanations for 208148_2
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.44
Dispersal of feature importance: 0.02
Category 1 of 4 . Testing 7 of 10 .
Generating local Explanations for 205256_2
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.65
Dispersal of feature importance: 0.32
Category 1 of 4 . Testing 8 of 10 .
Generating local Explanations for 211275_2
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.61
Dispersal of feature importance: 0.09
Category 1 of 4 . Testing 9 of 10 .
Generating local Explanations for 205340_2
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.62
Dispersal of feature importance: 0.04
Category 1 of 4 . Testing 8 of 10 .
Generating local Explanations for 208055_3
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.01
Category 1 of 4 . Testing 9 of 10 .
Generating local Explanations for 211543_3
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.57
Dispersal of feature importance: 0.03
Category 1 of 4 . Testing 10 of 10 .
Generating local Explanations for 207674_3
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.75
Dispersal of feature importance: 0.18
Category 2 of 4 . Testing 1 of 10 .
Generating local Explanations for 213693_3
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.66
Dispersal of feature importance: 0.01
Category 2 of 4 . Testing 2 of 10 .
Generating local Explanations for 207278_3
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.36
Dispersal of feature importance: 0.09
Category 2 of 4 . Testing 1 of 10 .
Generating local Explanations for 205661_4
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.34
Dispersal of feature importance: 0.27
Category 2 of 4 . Testing 2 of 10 .
Generating local Explanations for 205167_4
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.3
Dispersal of feature importance: 0.14
Category 2 of 4 . Testing 3 of 10 .
Generating local Explanations for 205421_4
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.31
Dispersal of feature importance: 0.51
Category 2 of 4 . Testing 4 of 10 .
Generating local Explanations for 209913_4
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.32
Dispersal of feature importance: 0.17
Category 2 of 4 . Testing 5 of 10 .
Generating local Explanations for 209592_4
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6


Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.46
Dispersal of feature importance: 0.71
Category 2 of 4 . Testing 4 of 10 .
Generating local Explanations for 213624_5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.51
Dispersal of feature importance: 0.26
Category 2 of 4 . Testing 5 of 10 .
Generating local Explanations for 207026_5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.43
Dispersal of feature importance: 0.24
Category 2 of 4 . Testing 6 of 10 .
Generating local Explanations for 214040_5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.51
Dispersal of feature importance: 0.05
Category 2 of 4 . Testing 7 of 10 .
Generating local Explanations for 206339_5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.51
Dispersal of feature importance: 0.48
Category 2 of 4 . Testing 8 of 10 .
Generating local Explanations for 205217_5
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6

...

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.74
Dispersal of feature importance: 0.11
Category 1 of 4 . Testing 6 of 10 .
Generating local Explanations for 205358_36
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.05
Category 1 of 4 . Testing 7 of 10 .
Generating local Explanations for 205143_36
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.75
Dispersal of feature importance: 0.07
Category 1 of 4 . Testing 8 of 10 .
Generating local Explanations for 205568_36
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Dispersal of feature importance: 0.02
Category 1 of 4 . Testing 9 of 10 .
Generating local Explanations for 206333_36
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Dispersal of feature importance: 0.03
Category 1 of 4 . Testing 10 of 10 .
Generating local Explanations for 205062_36
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5


Run 9
Stability: 0.81
Dispersal of feature importance: 0.11
Category 1 of 4 . Testing 9 of 10 .
Generating local Explanations for 207966_37
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Dispersal of feature importance: 0.01
Category 1 of 4 . Testing 10 of 10 .
Generating local Explanations for 206411_37
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.03
Category 2 of 4 . Testing 1 of 10 .
Generating local Explanations for 208565_37
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.74
Dispersal of feature importance: 0.02
Category 2 of 4 . Testing 2 of 10 .
Generating local Explanations for 211098_37
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.7
Dispersal of feature importance: 0.01
Category 2 of 4 . Testing 3 of 10 .
Generating local Explanations for 212226_37
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.71
Dispersa

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Dispersal of feature importance: 0.01
Category 2 of 4 . Testing 3 of 10 .
Generating local Explanations for 205313_38
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.79
Dispersal of feature importance: 0.02
Category 2 of 4 . Testing 4 of 10 .
Generating local Explanations for 206928_38
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Dispersal of feature importance: 0.01
Category 2 of 4 . Testing 5 of 10 .
Generating local Explanations for 213300_38
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.05
Category 2 of 4 . Testing 6 of 10 .
Generating local Explanations for 211098_38
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Dispersal of feature importance: 0.02
Category 2 of 4 . Testing 7 of 10 .
Generating local Explanations for 210689_38
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5


Run 9
Stability: 0.77
Dispersal of feature importance: 0.01
Category 2 of 4 . Testing 6 of 10 .
Generating local Explanations for 211074_39
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Dispersal of feature importance: 0.01
Category 2 of 4 . Testing 7 of 10 .
Generating local Explanations for 210272_39
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.82
Dispersal of feature importance: 0.02
Category 2 of 4 . Testing 8 of 10 .
Generating local Explanations for 204751_39
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.02
Category 2 of 4 . Testing 9 of 10 .
Generating local Explanations for 208832_39
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.78
Dispersal of feature importance: 0.03
Category 2 of 4 . Testing 10 of 10 .
Generating local Explanations for 213088_39
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal

Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.81
Dispersal of feature importance: 0.02
Category 2 of 4 . Testing 10 of 10 .
Generating local Explanations for 209913_40
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.77
Dispersal of feature importance: 0.01
Category 3 of 4 . Testing 1 of 8 .
Generating local Explanations for 208103_40
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.68
Dispersal of feature importance: 0.13
Category 3 of 4 . Testing 2 of 8 .
Generating local Explanations for 206615_40
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.8
Dispersal of feature importance: 0.14
Category 3 of 4 . Testing 3 of 8 .
Generating local Explanations for 211098_40
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 6
Run 7
Run 8
Run 9
Stability: 0.74
Dispersal of feature importance: 0.15
Category 3 of 4 . Testing 4 of 8 .
Generating local Explanations for 208421_40
Run 0
Run 1
Run 2
Run 3
Run 4
Run 5
Run 